# Amortized Analysis

Amortized analysis is a method to analyze the time complexity of executing $n$ operations on a data structure, where we are interested in the overall performce on the $n$ operations, not just on individual operation, for some operations may require much more time to process than the others. Amortized analysis spreads out the cost among all operations.

# Idea

Given a sequence of operations on a data structure $D$, we spread out the cost (runtime) over the sequence.

1. Divide some of the cost of inefficient operations across the more efficient operations. This approach would make sense if the inefficient operations occur relatively infrequently within the sequence of operations to be analyzed. In other words, the cost of infrequent operations may be "shared" by efficient operations, providing an average cost of each operation.
		
2. Using the business definition of amortize is another way to think about this process. For example, loans are traditionally amortized, so that one pays the same amount per month over the life of the loan. In reality, in each month, different amounts of money are going to the principal and the interest.
		

# Three Methods

Amortized analysis is an analysis technique (In
	reality we cannot divide time among operations!).
    Namely, there is no change in computational model.
	

There are three main methods people use to do amortized analysis:

1. Aggregate method: Compute a simple mean.

2. Accounting method: Maintain a bank account of credits to pay for the operations.
		
3. Potential method: Give data structures potential energy similar to the notion of potential energy in physics.
		

# Aggregate Method

The aggregate method is the most intuitive form of amortized analysis.

1. In aggregate analysis we show that a sequence of $n$ operations takes worst-case time $T(n)$ in total. To get the amortized time, simply compute $T(n)/n$, the average time per operation. Note that we assign the <b>same</b> cost to every operation.
	

2. Examples. Let's try out this method on two data structures: stack and binary counter.


## Stack

The main operations on a stack are Push and Pop.

1. Push and Pop are local operations, and so the worst-case time is $O(1)$.
	
2. Given a sequence of $n$ operations consisting of Push and Pop, the worst-case time $T(n) = \sum_{i = 1}^n O(1) = O(n)$. Thus, the amortized time per operation is $O(1)$

Now consider a more expensive operation on a stack: Multipop. It takes an argument $k$ (the number of items to pop) as input and executes the Pop operation on the stack $k$ times without pusp in between, until the stack is empty.

1. Let $S$ denote a stack and $|S|$ its size. The worst-case runtime of Multipop(S,k) is $\min\{k, |S|\}$.

2. Consider three operations: Push, Pop, and Multipop.

## Pops Cannot Exceed Pushes

1. We should capitalize more on what we know about a sequence of $n$ stack operations starting from an empty stack.

2. Observation: An item can only be popped from the stack once.

3. Claim: A sequence of $n$ operations, on an initially empty stack, takes $O(n)$ time to run: The number of Pop operations (including Multipop) is at most the number of Push operations. There are, at most $n$ Push operations so there are at most $n$ Pop operations performed for a sequence of $n$ operations.
		
3. This means the amortized cost per operation is $O(n)/n = O(1)$.		
	

## Binary Counter

A binary counter has $k$-bits for a fixed $k$, similar to the $8$-digit odometer in your car.

1. Increment is the only operation.

2. Since we are restricted to $k$-bits, the counter can "rollover" (overflow).
	
We implement the binary counter as an array of size $k$ as follows:
	

In [5]:
def Increment(C): # C is a counter of k bits
    i = 0
    while (i < k) and (C[i] == 1):
        C[i] = 0
        i += 1
    if i < k: # carry bit
        C[i] = 1


Set the array to 0s initially. Want to find out the runtime of executing $n$ increments.

<b>Observation</b>:

1. While increment $i+1$ causes a number of bits to flip, not every bit flips every time.

2. Consider the following binary counter sequence ($k = 4$) with $C[0]$ being in the right-most column:

\begin{align*}
\begin{array}{cccc}
0 & 0 & 0 & 0\\
0 & 0 & 0 & \mathbf{1} \\
0 & 0 & \mathbf{1} & \mathbf{0}\\
0 & 0 & 1 & \mathbf{1} \\
0 & \mathbf{1} & \mathbf{0} & \mathbf{0}\\
0 & 1 & 0 & \mathbf{1}\\
0 & 1 & \mathbf{1} & \mathbf{0}\\
0 & 1 & 1 & \mathbf{1}\\
\mathbf{1} & \mathbf{0} & \mathbf{0} & \mathbf{0}\\
1 & 0 & 0 & \mathbf{1}\\
1 & 0 & \mathbf{1} & \mathbf{0}\\
1 & 0 & 1 & \mathbf{1} \\
1 & \mathbf{1} & \mathbf{0} & \mathbf{0}\\
1 & 1 & 0 & \mathbf{1}\\
1 & 1 & \mathbf{1} & \mathbf{0}\\
1 & 1 & 1 & \mathbf{1}\\
\mathbf{0} & \mathbf{0} & \mathbf{0} & \mathbf{0}
\end{array}
\end{align*}


## Bit Flips

1. In a sequence of $n$ operations, $C[0]$ flips every operation.

2. $C[1]$ flips every other operation; i.e., $\lfloor\frac{n}{2}\rfloor$ in $n$ operations.

3. $C[2]$ flips every $2^2 = 4$ operations.

4. Similarly, $C[i]$ flips every $2^i$ operations.

Thus,
\begin{align*}
\begin{array}{ll}
T(n) &= \sum_{i = 0}^{k-1}\left\lfloor \frac{n}{2^i}\right\rfloor & (\text{Change of indices})\\
&\leq \sum_{i = 0}^\infty \frac{n}{2^i} & (\text{Upperbound series})\\
&= n\sum_{i = 0}^\infty \left(\frac{1}{2}\right)^i & (\text{Geometric series})\\
&= n\left(\frac{1}{1 - \frac{1}{2}}\right) & \\
&= 2n &
\end{array}
\end{align*}

5. Taking an average to get $2n/n = 2$ amortized time per Increment operation.	


# Accounting Method

The accounting method is a business way of thinking about amortized analysis:
	
1. Assign every operation an amortized cost in credits. This may be more than or less than the actual cost, often referred to as the <b>charge</b> for the operation.
		
2. We put any overages in a "piggy bank". An overage occurs when the amortized cost of an operation exceeds the actual cost of an operation.
		
3. If we don't have enough credits, from the amortized cost, to cover the actual cost of the operation we have a <b>deficit</b>. To cover the deficit, we extract the required credits from the piggy bank.
			
4. How to assign amortized costs to operations is important. It needs to guarantee that regardless of the sequence of operations the amortized costs will be an upper bound of the actual cost.
	

## Example 1: Stack with Multipop
	
1. To use the accounting method we must assign an appropriate amortized cost to each operation.

\begin{align*}
\begin{array}{|l|c|c|}
    \hline
	\textbf{Operation} & \textbf{Actual Cost} & \textbf{Amortized Cost}\\
    \hline
	\text{Push}	   & 1			 & 2\\
	\hline
	\text{Pop}	   & 1 			 & 0\\
    \hline
	\text{Multipop}  & \min\{k, |S|\} & 0 \\
    \hline
\end{array}
\end{align*}

2. That is, we think ahead: For each element pushed into the stack, we will also budget the cost of popping it to the push, leaving the cost of element-popping for free.


## Does It Work?

		
1. It does.
2. One can only pop elements that have been pushed onto the stack.
3. Regardless of which Multipush and Push is used, removing an item requires constant time.
4. Each item pushed to the stack charges 1 credit, which leaves 1 credit for the piggy bank to pop the item being pushed. Every pop require us to take one credit out of the piggy bank.
			
5. This means a sequence of $n$ operations costs $O(n)$ time, implying an amortized cost of $O(1)$ for every operation.

## Binary Counter

Need to think of the two sub-operations for Increment:
		
1. Set bit $C[i]$; i.e., set bit $C[i]$ to 1. The actual cost of this operation is 1.
			
2. Clear bit $C[i]$; i.e., set bit $C[i]$ to 0. The actual cost of this operation is 1.
			
3. What can we do for amortized cost? The set-bit operation will have amortized cost of 2, one for setting it, and one for clearing it.

4. Note: A bit cannot be cleared without being set first.

5. The clear-bit operation will have amortized cost 0.

6. Setting a bit is charged 2 credits, one for the operation and one for the piggy bank.

7. Clearing a bit uses the one credit overcharged for setting the bit from the piggy bank.

8. Since we can never have a negative number of 1s, the balance of the piggy bank is never below zero.
			
9. The cost of clearing the bits in the <b>while</b> loop for Increment is covered by the money in the piggy bank.

10. The cost to set a bit at the index of Increment requires a cost of 2.

Taking these facts together we arrive at a cost of $2n$ for $n$ Increment operations,
thus the average amortized cost of an Increment operation is $2$.
			
		
		

# Potential Method

The potential meethod is the most general method. It looks at a data structure and measure its "energy change" in each operation from the previous one.
	
1. Assign a <b>potential energy</b> to a given state of the data structure.
		
2. Every operation puts the data structure into a different state that either increases or decreases the potential energy.

3. Denote by $D_i$ the state of a data structure after the $i$-th operation.

4. Denote by $\Phi(D_i)$ the potential energy of the data structure in state $D_i$. This is a real value. The function $\Phi$ is referred to as the potential function.
		
5. Denote the amortized cost of the $i$-th operation by $\hat{c}_i$. Formally, 

$$\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i - 1}),$$ 

where $c_i$ is the actual cost of the $i$-th operation.
	

## The Total Amortized Cost for $n$ Operations

\begin{align*}
\sum_{i = 1}^n \hat{c}_i &= \sum_{i = 1}^n (c_i + \Phi(D_i) - \Phi(D_{i -1})) &\\
&= \sum_{i = 1}^n c_i + \sum_{i = 1}^n (\Phi(D_i) - \Phi(D_{i -1})) & (\text{Telescoping series.})\\
&= \sum_{i = 1}^n c_i + \Phi(D_n) - \Phi(D_0).
\end{align*}
    
1. If $\Phi(D_n) \geq \Phi(D_0)$, then the total amortized cost is an upper bound of the total actual cost.
		
2. In practice, $\Phi(D_i) \geq \Phi(D_0)$ for all $i$. This guarantees we store up enough energy in advance. Typically, $\Phi(D_0) = 0$.

## Potential Functions

There are a wide range of potential functions that can be used for the analysis.
The choice depends on how tight of a bound we want.
	
Let's look at the stack again.
	
1. Define the potential function to be the number of items in stack $S$. This naturally gives us $\Phi(D_0) = 0$. Notice that stacks cannot have negative potential.
		
2. $\Phi(D_i) \geq 0$ for all $i$.

3. The fact $\Phi(D_i) \geq 0$ for all $i$ means that $\Phi(D_i) \geq \Phi(D_0)$ for all $i$. Thus conserving sufficient energy.
		

## Amortized Cost of Stack Operations

1. Push has an amortized cost 2. Suppose the $i$-th operation is a Push, then

$$\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i - 1}) = 1 + 1 = 2.$$


2. Multipop has an amortized cost 0. 

$$\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i - 1}) = \min\{|D_{i-1}|, k\} + (|D_{i-1}| -
				\min\{|D_{i-1}|, k\}) - |D_{i-1}| = 0.$$



3. Pop has an amortized cost 0.

$$\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i - 1}) = 1 - 1 = 0.$$

## Binary Counter

1. Let $\Phi(D_i)$ be the number of 1s in the counter after the $i$-th Increment operation.
2. We have $\Phi(D_0) = 0$ since we start the counter with 0s.
3. Let $t_i$ be the number of bits reset from 1 to 0 by the $i$-th Increment operation. This means the actual cost $c_i$ is $t_i + 1$ since we have to set one bit.
4. (1) If $\Phi(D_i) = 0$, then it means that all bits are reset from 1 to 0. This implies $\Phi(D_{i - 1}) = t_i = k$ (only when the binary counter was full can this happen). (2) If $\Phi(D_i) > 0$, then $\Phi(D_i) = \Phi(D_{i - 1}) - t_i + 1$.

5. The potential difference for the $i$-th Increment operation is $\Phi(D_i) - \Phi(D_{i-1}) = 1 - t_i$.
		
6. The amortized cost of Increment is $(t_i + 1) + (1 - t_i) = 2$.

Note: We can use the potential method to argue that the bound holds even if the counter does not start at zero.
